# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-5665f0cba1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Monarchies") 

Consider the following exploratory information need:

> You investigating existing and past historical monarchies

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P1366`   | replaced by    | predicate |
| `wdt:P122`    | basic form of government    | predicate |
| `wd:Q38`      | Italy          | node |
| `wd:Q145`     | United Kingdom           | node |
| `wd:Q7269`    | monarchy           | node |




Also consider

```
?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
```

is the BGP to retrieve all **countries that are some instance of a monarchy**

## Workload Goals


1. Identify the BGP to retrieve states that were monarchies and have been replaced by some other state

2. Identify the BGP for obtaining statistical information of monarchies, e.g., poluation or inception date

3. Which is or was  the country with the highest population?

4. How many countries are or have been monarchies across continents?

5. Analyze the number of countries that were monarchies
 
   5.1 How many sub-types of monarchy exist?  How many instances of exist of each one?
   
   5.2 Are there cities that have been capital of multiple kingdoms?
   
   5.3 Which are the top-3 monarchies with the earliest inception and still existing?


#### Example of query

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2344')]


1

### My Workflow

#### ***Task 1 :*** Identify the BGP to retrieve states that were monarchies and have been replaced by some other state.

I show some examples of countries that are some instance of a monarchy.

In [3]:
queryString = """
SELECT DISTINCT ?monarchy ?monarchyName WHERE { 

    # All countries that are some instance of a monarchy
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279* wd:Q7269  .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q139315'), ('monarchyName', 'Dʿmt')]
[('monarchy', 'http://www.wikidata.org/entity/Q688859'), ('monarchyName', 'Duchy of Bouillon')]
[('monarchy', 'http://www.wikidata.org/entity/Q15883591'), ('monarchyName', 'Anhalt-Dessau')]
[('monarchy', 'http://www.wikidata.org/entity/Q2308237'), ('monarchyName', 'Ahom Kingdom')]
[('monarchy', 'http://www.wikidata.org/entity/Q3919363'), ('monarchyName', 'Principality of Lahojsk')]
[('monarchy', 'http://www.wikidata.org/entity/Q6207845'), ('monarchyName', 'Jodhpur State')]
[('monarchy', 'http://www.wikidata.org/entity/Q1740533'), ('monarchyName', 'Baku Khanate')]
[('monarchy', 'http://www.wikidata.org/entity/Q13032661'), ('monarchyName', 'Principality of Slutsk')]
[('monarchy', 'http://www.wikidata.org/entity/Q625380'), ('monarchyName', 'Eastern Hungarian Kingdom')]
[('monarchy', 'http://www.wikidata.org/entity/Q896856'), ('monarchyName', 'Prince-Bishopric Konstanz')]


10

I try to see what is connected to monarchies through the property ***replaced by (wdt:P1366)***.

In [4]:
queryString = """
SELECT DISTINCT ?o ?oName WHERE { 

    # Connect monarchies to something using the property replaced by (wdt:P1366)
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269  ;
                wdt:P1366                     ?o        .

    # This returns the labels
    ?o <http://schema.org/name> ?oName .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8959'), ('oName', 'Northern Yuan Dynasty')]
[('o', 'http://www.wikidata.org/entity/Q15903902'), ('oName', 'Da Ming Shun Tian Guo')]
[('o', 'http://www.wikidata.org/entity/Q7418370'), ('oName', 'Sanjak of Herzegovina')]
[('o', 'http://www.wikidata.org/entity/Q153080'), ('oName', 'East Francia')]
[('o', 'http://www.wikidata.org/entity/Q693356'), ('oName', 'Hesse-Nassau')]
[('o', 'http://www.wikidata.org/entity/Q1745385'), ('oName', 'United States of Matsya')]
[('o', 'http://www.wikidata.org/entity/Q202311'), ('oName', 'Egypt')]
[('o', 'http://www.wikidata.org/entity/Q958291'), ('oName', 'United Principalities')]
[('o', 'http://www.wikidata.org/entity/Q1998459'), ('oName', 'Free State of Mecklenburg-Schwerin')]
[('o', 'http://www.wikidata.org/entity/Q153402'), ('oName', 'Roer')]
[('o', 'http://www.wikidata.org/entity/Q651377'), ('oName', 'Shunga Empire')]
[('o', 'http://www.wikidata.org/entity/Q139319'), ('oName', 'Russian Republic')]
[('o',

20

So, probably the BGP to retrieve states that were monarchies and have been replaced by some other state is:
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
            wdt:P1366                     ?replacingState .
```

In [5]:
queryString = """
SELECT DISTINCT ?monarchyName ?replacingStateName WHERE { 

    # All monarchies that have been replaced by another state
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
                wdt:P1366                     ?replacingState .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
    ?replacingState <http://schema.org/name> ?replacingStateName .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchyName', 'Jodhpur State'), ('replacingStateName', 'India')]
[('monarchyName', 'Eastern Hungarian Kingdom'), ('replacingStateName', 'Principality of Transylvania')]
[('monarchyName', 'East Francia'), ('replacingStateName', 'Holy Roman Empire')]
[('monarchyName', 'Kingdom of Hawaiʻi'), ('replacingStateName', 'Republic of Hawaii')]
[('monarchyName', 'Duchy of Luxembourg'), ('replacingStateName', 'French First Republic')]
[('monarchyName', 'Taifa of Algeciras'), ('replacingStateName', 'Taifa of Seville')]
[('monarchyName', 'Kotoko kingdom'), ('replacingStateName', 'Bornu Empire')]
[('monarchyName', 'Province of North Carolina'), ('replacingStateName', 'North Carolina')]
[('monarchyName', 'Sui dynasty'), ('replacingStateName', 'Tang Empire')]
[('monarchyName', 'Song dynasty'), ('replacingStateName', 'Yuan Empire')]
[('monarchyName', 'United Principalities'), ('replacingStateName', 'Kingdom of Romania')]
[('monarchyName', 'Travancore'), ('replacingStateName', 'India')]
[('mo

20

The BGP was correct, but what if I want to show only the "last replacing state" ? 

I can extend the path repeating one or more time the property ***replaced by (wdt:P1366)***. 

I have also to add a ***NOT EXISTS { ?finalReplacingState wdt:P1366 ?replacement }*** to have only a "last replacing state" that has not been replaced by any other state.

In [6]:
queryString = """
SELECT DISTINCT ?monarchyName ?finalReplacingStateName WHERE { 

    # All monarchies that have been replaced (at least one time) by another state
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
                wdt:P1366+                    ?finalReplacingState .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
    ?finalReplacingState <http://schema.org/name> ?finalReplacingStateName .
    
    # Ensuring that the finalReplacingState have not been replaced by other states
    FILTER NOT EXISTS { ?finalReplacingState wdt:P1366 ?replacement }    
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('monarchyName', 'Mataram Sultanate'), ('finalReplacingStateName', 'Yogyakarta Sultanate')]
[('monarchyName', 'Mataram Sultanate'), ('finalReplacingStateName', 'Surakarta Sunanate')]
[('monarchyName', 'Kingdom of Jerusalem'), ('finalReplacingStateName', 'Mamluk Sultanate')]
[('monarchyName', 'Prince-Bishopric of Liège'), ('finalReplacingStateName', 'French Fifth Republic')]
[('monarchyName', 'Kamakura shogunate'), ('finalReplacingStateName', 'Japan')]
[('monarchyName', 'Western Han'), ('finalReplacingStateName', 'Northern Yuan Dynasty')]
[('monarchyName', 'Western Han'), ('finalReplacingStateName', 'Tibetan Empire')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Grand Duchy of Hesse')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Ostmark')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Germany')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Second Czechoslovak Republic')]
[('monarchyN

20

So the the BGP to retrieve only the "last replacing state" of a monarchy is: 
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
            wdt:P1366+                    ?finalReplacingState .
            
FILTER NOT EXISTS { ?finalReplacingState wdt:P1366 ?replacement }
```

However some of the replacing states are still monarchies. What if I want only states that were monarchies and that were finally replaced by non-monarchical states?

I need to use another ***NOT EXIST*** to ensure that the "last replacing state" is not a monarchy.

In [7]:
queryString = """
SELECT DISTINCT ?monarchyName ?finalReplacingStateName WHERE { 

    # All monarchies that have been replaced (at least one time) by another state
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
                wdt:P1366+                    ?finalReplacingState .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
    ?finalReplacingState <http://schema.org/name> ?finalReplacingStateName .
    
    # Ensuring that the final Replacing States have not been replaced by other states
    FILTER NOT EXISTS { ?finalReplacingState wdt:P1366 ?replacement }
    
    # Ensuring that the final Replacing States are not monarchies
    FILTER NOT EXISTS { ?finalReplacingState (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269 }
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchyName', 'Mataram Sultanate'), ('finalReplacingStateName', 'Surakarta Sunanate')]
[('monarchyName', 'Prince-Bishopric of Liège'), ('finalReplacingStateName', 'French Fifth Republic')]
[('monarchyName', 'Western Han'), ('finalReplacingStateName', 'Northern Yuan Dynasty')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Ostmark')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Germany')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Second Czechoslovak Republic')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Slovenia')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'North Macedonia')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Croatia')]
[('monarchyName', 'Holy Roman Empire'), ('finalReplacingStateName', 'Montenegro')]


10

Hence, in order to retrieve only states that were monarchies and that were finally replaced by non-monarchical states I can use this BGP:
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
            wdt:P1366+                    ?finalReplacingState .
            
FILTER NOT EXISTS { ?finalReplacingState wdt:P1366 ?replacement }           
FILTER NOT EXISTS { ?finalReplacingState (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269 }
```

#### END TASK 1: 
I was able to retrieve the BGP to retrieve states that were monarchies and have been replaced by some other state is:
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
            wdt:P1366                     ?replacingState .
```

Moreover, it is also possible to add other FILTER in order to obtain different types of information (such as the "last replacing state").

### ***Task 2 :*** Identify the BGP for obtaining statistical information of monarchies, e.g., populuation or inception date

As a starting point, I want to retrieve a URI for a specific monarchy, so I try to use the property ***replaced by (wdt:P1366)*** with ***Italy (wd:Q38)***.

In [8]:
queryString = """
SELECT DISTINCT ?monarchy ?monarchyName WHERE { 

    # Connecting a monarchy to Italy (wd:Q38) through the property replaced by (wdt:P1366)
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269  ;
                wdt:P1366                     wd:Q38    .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
}
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q172579'), ('monarchyName', 'Kingdom of Italy')]


1

I retrieve all the data properties of ***Kingdom of Italy (wd:Q172579)***.

In [9]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Connecting Kingdom of Italy to something
    wd:Q172579  ?p  ?o.

    # This returns the labels
    ?p <http://schema.org/name> ?pName .

    # Only data properties
    FILTER(isLiteral(?o))
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pName', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('pName', 'official name')]
[('p', 'http://www.wikidata.org/prop/direct/P1549'), ('pName', 'demonym')]
[('p', 'http://www.wikidata.org/prop/direct/P1711'), ('pName', 'British Museum person or institution ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pName', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pName', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4839'), ('pName', 'Wolfram Language entity code')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'in

20

I discovered the URI both of:
* ***population (wdt:P1082)***
* ***inception (wdt:P571)***

I want to check what is connected to ***Kingdom of Italy (wd:Q172579)*** through these two property.

I start with ***population (wdt:P1082)***.

In [10]:
queryString = """
SELECT ?population WHERE { 

    # Retrieve population of Kingdom of Italy
    wd:Q172579  wdt:P1082  ?population.
}
"""

print("Results")
run_query(queryString)

Results
[('population', '22182000')]
[('population', '35845000')]
[('population', '42399000')]


3

The property is correct but there are multiple results. Maybe there is a subproperty of ***population (wdt:P1082)*** that gives additional information.

In [11]:
queryString = """
SELECT ?subproperty WHERE { 

    # Retrieve a subproperty of property population
    ?subproperty wdt:P1647 wdt:P1082 .
}
"""

print("Results")
run_query(queryString)

Results
Empty


0

There is no subproperty. Maybe Wikidata manage these kinds of this information in another way, and so we can't access them.

Now I try the property ***inception (wdt:P571)*** with ***Kingdom of Italy (wd:Q172579)***.

In [12]:
queryString = """
SELECT ?inceptionDate WHERE { 

    # Retrieve inceptionDate of Kingdom of Italy
    wd:Q172579  wdt:P571  ?inceptionDate .
}
"""

print("Results")
run_query(queryString)

Results
[('inceptionDate', '1861-03-17T00:00:00Z')]


1

It works, so the BGP to retrieve inceptionDate of monarchies is :
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269       ;
            wdt:P571                      ?inceptionDate . 
```

In [13]:
queryString = """
SELECT DISTINCT ?monarchyName ?inceptionDate WHERE { 

    # Retrieve inceptionDate of monarchies
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269       ;
                wdt:P571                      ?inceptionDate . 

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchyName', 'Dʿmt'), ('inceptionDate', '-979-01-01T00:00:00Z')]
[('monarchyName', 'Ahom Kingdom'), ('inceptionDate', '1228-01-01T00:00:00Z')]
[('monarchyName', 'Jodhpur State'), ('inceptionDate', '1250-01-01T00:00:00Z')]
[('monarchyName', 'Baku Khanate'), ('inceptionDate', '1735-01-01T00:00:00Z')]
[('monarchyName', 'Eastern Hungarian Kingdom'), ('inceptionDate', '1526-11-21T00:00:00Z')]
[('monarchyName', 'Duchy of the Archipelago'), ('inceptionDate', '1207-01-01T00:00:00Z')]
[('monarchyName', 'Kingdom of Sicily'), ('inceptionDate', '1848-03-25T00:00:00Z')]
[('monarchyName', 'East Francia'), ('inceptionDate', '0843-01-01T00:00:00Z')]
[('monarchyName', 'Kingdom of Hawaiʻi'), ('inceptionDate', '1795-01-01T00:00:00Z')]
[('monarchyName', 'Duchy of Luxembourg'), ('inceptionDate', '1353-01-01T00:00:00Z')]


10

#### END TASK 2: 
I was able to retrieve the BGP to retrieve statistical information about monarchies :
```
?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269       ;
            wdt:P571                      ?inceptionDate ;
            wdt:P1082                     ?population    .
```

However, when the property ***population (wdt:P1082)*** is used, multiple values are returned. 

This behaviour is due to the fact that Wikidata stores some additional information using ***property qualifiers*** that make use of the ***namespace pq***. 

Since we can't use the namespace pq in our notebook it's not possibile to retrieve some additional information. (such as when the population values were calculated)


### ***Task 3 :*** Which is or was the country with the highest population?

To inspect population of different countries, I need the URI to represent a Country. 

To do this I use the property ***instanceOf (wdt:P31)*** with ***Italy (wd:Q38)***.

In [14]:
queryString = """
SELECT ?class ?className WHERE { 

    # Use theh property instanceOf together with Italy
    wd:Q38 wdt:P31 ?class .
    
    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3624078'), ('className', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q6256'), ('className', 'country')]


2

I can use ***Country(wd:Q6256)*** together with the property ***population (wdt:P1082)*** to find the country with the highest population

In [15]:
queryString = """
SELECT ?country ?countryName ?population WHERE { 

    # Retrieve countries and their population
    ?country  wdt:P31    wd:Q6256    ;
              wdt:P1082  ?population .
    
    # This returns the labels
    ?country <http://schema.org/name> ?countryName .
}
ORDER BY DESC (?population)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China"), ('population', '1443497378')]


1

#### END TASK 3: 
I discovered that the ***Country(wd:Q6256)*** with the highest population is ***People's Republic of China (wd:Q148)***.

### ***Task 4 :*** How many countries are or have been monarchies across continents?
First, I have to understand how Countries are related to Continents. To do this, I can retrieve all the object properties of ***Kingdom of Italy (wd:Q172579)***.

In [16]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Connecting Kingdom of Italy to something
    wd:Q172579  ?p  ?o.

    # This returns the labels
    ?p <http://schema.org/name> ?pName .

    # Exclude data properties
    FILTER(!isLiteral(?o))
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pName', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pName', 'flag')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P237'), ('pName', 'coat of arms')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('pName', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used')]
[('p', 'http://www.wikidata.org/prop/direc

20

Probably, I can use the property ***continent (wdt:P30)***. Let'see what is connected to ***Kingdom of Italy (wd:Q172579)*** through this property.

In [17]:
queryString = """
SELECT DISTINCT ?continent ?continentName WHERE { 

    # Connecting Kingdom of Italy using property Continent
    wd:Q172579  wdt:P30 ?continent .

    # This returns the labels
    ?continent <http://schema.org/name> ?continentName .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe')]


1

I can retrieve all the monarchies and divide them by continent.

In [18]:
queryString = """
SELECT ?continent ?continentName ?numMonarchiesByContinent WHERE { 

    # Retrieve the number of monarchies for each continent
    {  SELECT ?continent COUNT(DISTINCT ?monarchy) AS ?numMonarchiesByContinent WHERE {
        
        # Retrieve the continent of each monarchy
        ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269   ;  
                    wdt:P30                       ?continent .
        }
        GROUP BY ?continent
    }
    
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName . 
}
ORDER BY DESC(?numMonarchiesByContinent)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('numMonarchiesByContinent', '368')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('numMonarchiesByContinent', '175')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('numMonarchiesByContinent', '68')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('numMonarchiesByContinent', '31')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('numMonarchiesByContinent', '14')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('numMonarchiesByContinent', '13')]
[('continent', 'http://www.wikidata.org/entity/Q107665732'), ('continentName', 'Hyboria'), ('numMonarchiesByContinent', '13')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('numMonarchiesByContinent', '8')]
[('con

16

Ok, since there are some strange continents, I want to discover more about them. I use the property ***instanceOf (wdt:P31)*** on the retrieved continents.

In [19]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Retrieve the continent of each monarchy
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269   ;  
                wdt:P30                       ?continent .

    # Use the property instanceOf on the continents
    ?continent  wdt:P31 ?class .
    
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName .
    ?class <http://schema.org/name> ?className .

}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q312461'), ('className', 'terra nullius')]
[('class', 'http://www.wikidata.org/entity/Q33837'), ('className', 'archipelago')]
[('class', 'http://www.wikidata.org/entity/Q5107'), ('className', 'continent')]
[('class', 'http://www.wikidata.org/entity/Q82794'), ('className', 'geographic region')]
[('class', 'http://www.wikidata.org/entity/Q17199338'), ('className', 'fictional continent')]
[('class', 'http://www.wikidata.org/entity/Q55833'), ('className', 'supercontinent')]
[('class', 'http://www.wikidata.org/entity/Q2418896'), ('className', 'part of the world')]
[('class', 'http://www.wikidata.org/entity/Q855697'), ('className', 'subcontinent')]


8

I discovered that there are also some fictional continent and supercontinent. If I want only the "traditional" ***continents (wd:Q5107)*** I have to add this:
```
?continent wdt:P31 wd:Q5107 .
```

Now I can repeate the previous query (retrieve all the monarchies and divide them by continent) but using only "traditional" continents.

In [20]:
queryString = """
SELECT ?continent ?continentName ?numMonarchiesByContinent WHERE { 

    # Retrieve the number of monarchies for each continent
    {  SELECT ?continent COUNT(DISTINCT ?monarchy) AS ?numMonarchiesByContinent WHERE {
        
        # Retrieve the continent of each monarchy
        ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269   ;  
                    wdt:P30                       ?continent .
        
        # I want only "traditional" continent
        ?continent   wdt:P31 wd:Q5107
        }
        GROUP BY ?continent
    }
    
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName . 
}
ORDER BY DESC(?numMonarchiesByContinent)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('numMonarchiesByContinent', '368')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('numMonarchiesByContinent', '175')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('numMonarchiesByContinent', '68')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('numMonarchiesByContinent', '31')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('numMonarchiesByContinent', '13')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('numMonarchiesByContinent', '8')]
[('continent', 'http://www.wikidata.org/entity/Q828'), ('continentName', 'Americas'), ('numMonarchiesByContinent', '5')]
[('continent', 'http://www.wikidata.org/entity/Q3960'), ('continentName', 'Australia'), ('numMonarchiesByContinent', '2')]
[('continen

9

#### END TASK 4: 
With this final query is possible to retrieve how many countries are or have been monarchies across continents.

However Wikidata is not so consistant when using ***continent (wd:Q5107)*** because of the different conventions to identify continents.

### ***Task 5 :*** Analyze the number of countries that were monarchies.
In this case I want only the number of countries that replaced a monarchy, so I should use the property ***replaced by (wdt:P1366)*** on all the instanced of monarchy. 

I have also to ensure that the replacing state is not a monarchy.

In [21]:
queryString = """
SELECT COUNT (DISTINCT ?monarchy) AS ?numCountries WHERE { 

    # All monarchies that have been replaced (at least one time) by another state
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269        ;
                wdt:P1366                     ?replacingState .
   
    # Ensuring that the replacing states are not monarchies
    FILTER NOT EXISTS { ?replacingState (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269 }
}
"""

print("Results")
run_query(queryString)

Results
[('numCountries', '227')]


1

#### END TASK 5: 
There are 227 monarchies that were directly replaced by a non-monarchical country.

### ***Task 5.1 :*** How many sub-types of monarchy exist? How many instances exist of each one?

In order to to discover sub-types of monarchy, I use the property ***subclassOf (wdt:P279)*** together with the node ***monarchy (wd:Q7269)***.

In [22]:
queryString = """
SELECT DISTINCT ?subMonarchy ?subMonarchyName WHERE { 

    # Retrieve everything that is subClass of a monarchy
    ?subMonarchy  wdt:P279  wd:Q7269 .
   
    # This returns the labels
    ?subMonarchy <http://schema.org/name> ?subMonarchyName . 
}
"""

print("Results")
run_query(queryString)

Results
[('subMonarchy', 'http://www.wikidata.org/entity/Q15622229'), ('subMonarchyName', 'patrimonialism')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q4482688'), ('subMonarchyName', 'feudal monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q179352'), ('subMonarchyName', 'despotism')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q584683'), ('subMonarchyName', 'elective monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q849242'), ('subMonarchyName', 'hereditary monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q184558'), ('subMonarchyName', 'absolute monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q3208952'), ('subMonarchyName', 'federal monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q41614'), ('subMonarchyName', 'constitutional monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q154547'), ('subMonarchyName', 'duchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q417175'), ('subMonarchyName', 'kingdom'

26

I can recognize some types of monarchies (for example federal monarchy), but there are also some specific monarchies such as Monarchy of Liechtenstein. 

I want to clarify better this aspect relying on the property ***instanceOf (wdt:P31)***.



In [23]:
queryString = """
SELECT DISTINCT ?class ?className WHERE { 

    # Retrieve the class of everything that is subClass of a monarchy
    ?subMonarchy  wdt:P279  wd:Q7269 ;
                  wdt:P31   ?class .
   
    # This returns the labels
    ?class <http://schema.org/name> ?className . 
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q11514315'), ('className', 'historical period')]
[('class', 'http://www.wikidata.org/entity/Q1307214'), ('className', 'form of government')]
[('class', 'http://www.wikidata.org/entity/Q13406463'), ('className', 'Wikimedia list article')]
[('class', 'http://www.wikidata.org/entity/Q183039'), ('className', 'form of state')]
[('class', 'http://www.wikidata.org/entity/Q1371288'), ('className', 'vassal state')]
[('class', 'http://www.wikidata.org/entity/Q15617994'), ('className', 'designation for an administrative territorial entity')]
[('class', 'http://www.wikidata.org/entity/Q28108'), ('className', 'political system')]
[('class', 'http://www.wikidata.org/entity/Q18810687'), ('className', 'political institution')]


8

The most correct class seems to be ***form of government (wd:Q1307214)***. 

So, now I only retrieve sub-types of monarchy that are also istance of the class ***form of government (wd:Q1307214)***.

In [24]:
queryString = """
SELECT DISTINCT ?subMonarchy ?subMonarchyName WHERE { 

    # Retrieve form of government that are also subclasses of Monarchy
    ?subMonarchy  wdt:P279  wd:Q7269 ;
                  wdt:P31   wd:Q1307214 .
   
    # This returns the labels
    ?subMonarchy <http://schema.org/name> ?subMonarchyName .
}
"""

print("Results")
run_query(queryString)

Results
[('subMonarchy', 'http://www.wikidata.org/entity/Q15622229'), ('subMonarchyName', 'patrimonialism')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q4482688'), ('subMonarchyName', 'feudal monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q179352'), ('subMonarchyName', 'despotism')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q584683'), ('subMonarchyName', 'elective monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q849242'), ('subMonarchyName', 'hereditary monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q184558'), ('subMonarchyName', 'absolute monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q3208952'), ('subMonarchyName', 'federal monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q41614'), ('subMonarchyName', 'constitutional monarchy')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q208500'), ('subMonarchyName', 'principality')]
[('subMonarchy', 'http://www.wikidata.org/entity/Q187587'), ('subMonarchyName', 'e

13

I finally have a list of different sub-type of monarchy. Now I can count how many instances exist of each one.

In [25]:
queryString = """
SELECT ?subMonarchyName ?numIstances WHERE{

    # Retrieve the number of countries that are a subMonarchy
    {    SELECT ?subMonarchy COUNT(DISTINCT ?country) as ?numIstances WHERE { 

            # Retrieve countries that that are some instance of a subMonarchy
            ?country (wdt:P31|wdt:P122)/wdt:P279* ?subMonarchy .

            # Retrieve form of government that are subclasses of Monarchy
            ?subMonarchy  wdt:P279  wd:Q7269 ;
                          wdt:P31   wd:Q1307214 .
        }
        GROUP BY ?subMonarchy
    }
    
    # This returns the labels
    ?subMonarchy <http://schema.org/name> ?subMonarchyName .
    
}
ORDER BY DESC(?numIstances)
"""

print("Results")
run_query(queryString)

Results
[('subMonarchyName', 'principality'), ('numIstances', '837')]
[('subMonarchyName', 'constitutional monarchy'), ('numIstances', '201')]
[('subMonarchyName', 'absolute monarchy'), ('numIstances', '170')]
[('subMonarchyName', 'elective monarchy'), ('numIstances', '34')]
[('subMonarchyName', 'feudal monarchy'), ('numIstances', '24')]
[('subMonarchyName', 'hereditary monarchy'), ('numIstances', '22')]
[('subMonarchyName', 'empire'), ('numIstances', '20')]
[('subMonarchyName', 'federal monarchy'), ('numIstances', '7')]
[('subMonarchyName', 'composite monarchy'), ('numIstances', '4')]
[('subMonarchyName', 'patrimonialism'), ('numIstances', '2')]
[('subMonarchyName', 'despotism'), ('numIstances', '1')]


11

#### END TASK 5.1: 
With this final query I managed to count how many instances exist of each sub-type of monarchy. 

In my opinion ***empire*** is a form of ***absolute monarchy*** but in the results I have both of them probably because Wikidata subclasses are not often consistently used.

### ***Task 5.2 :*** Are there cities that have been capital of multiple kingdoms?

First of all I need to undestand how capitals are related to kingdoms.

Kingdoms are a specific type of monarchy and I have already retrieve this URI in a previous query: ***Kingdom (wd:Q417175)*** .

I start by showing some examples of Kingdom.

In [26]:
queryString = """
SELECT DISTINCT ?kingdom ?kingdomName WHERE { 

    # Retrieve Kingdoms
    ?kingdom (wdt:P31|wdt:P122) wd:Q417175 .
     
    # This returns the labels
    ?kingdom <http://schema.org/name> ?kingdomName .
    
}
LIMIT 10    
"""

print("Results")
run_query(queryString)

Results
[('kingdom', 'http://www.wikidata.org/entity/Q139315'), ('kingdomName', 'Dʿmt')]
[('kingdom', 'http://www.wikidata.org/entity/Q2308237'), ('kingdomName', 'Ahom Kingdom')]
[('kingdom', 'http://www.wikidata.org/entity/Q625380'), ('kingdomName', 'Eastern Hungarian Kingdom')]
[('kingdom', 'http://www.wikidata.org/entity/Q733668'), ('kingdomName', 'Ghassanids')]
[('kingdom', 'http://www.wikidata.org/entity/Q332137'), ('kingdomName', 'Kingdom of Commagene')]
[('kingdom', 'http://www.wikidata.org/entity/Q3359147'), ('kingdomName', 'Videha')]
[('kingdom', 'http://www.wikidata.org/entity/Q22338414'), ('kingdomName', 'Kingdom Saal')]
[('kingdom', 'http://www.wikidata.org/entity/Q153015'), ('kingdomName', 'Kingdom of Saxony')]
[('kingdom', 'http://www.wikidata.org/entity/Q756617'), ('kingdomName', 'Danish Realm')]
[('kingdom', 'http://www.wikidata.org/entity/Q1275158'), ('kingdomName', 'Elymais')]


10

Now I  show all the object properties of a specific Kingdom such as ***Kingdom of Saxony (wd:Q153015)***.

In [27]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Everything that is connected to Kingdom of Saxony
    wd:Q153015 ?p ?o.
           
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
    
    # Exclude data properties
    FILTER(!isLiteral(?o))
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pName', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P194'), ('pName', 'legislative body')]
[('p', 'http://www.wikidata.org/prop/direct/P242'), ('pName', 'locator map image')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pName', 'capital')]
[('p', 'http://www.wikidata.org/prop/direct/P37'), ('pName', 'official langua

19

I want to see what is connected through the property ***capital (wdt:P36)***.

In [28]:
queryString = """
SELECT DISTINCT ?capital ?capitalName WHERE { 

    # Connecting Kingdom of Saxony using the property capital
    wd:Q153015 wdt:P36 ?capital.
           
    # This returns the labels
    ?capital <http://schema.org/name> ?capitalName .
} 
"""

print("Results")
run_query(queryString)

Results
[('capital', 'http://www.wikidata.org/entity/Q1731'), ('capitalName', 'Dresden')]


1

It works as expected. I want to retrieve the URI for a City so I use ***istanceOf (wdt:P31)*** with ***Dresden (wd:Q1731)***.

In [29]:
queryString = """
SELECT DISTINCT ?class?className WHERE { 

    # Connecting Dresden using the property istanceOf
    wd:Q1731 wdt:P31 ?class.
           
    # This returns the labels
    ?class <http://schema.org/name> ?className .
} 
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q14784328'), ('className', 'state capital in Germany')]
[('class', 'http://www.wikidata.org/entity/Q1549591'), ('className', 'big city')]
[('class', 'http://www.wikidata.org/entity/Q253030'), ('className', 'major regional center')]
[('class', 'http://www.wikidata.org/entity/Q42744322'), ('className', 'urban municipality of Germany')]
[('class', 'http://www.wikidata.org/entity/Q515'), ('className', 'city')]
[('class', 'http://www.wikidata.org/entity/Q61708099'), ('className', 'urban district in Saxony')]


6

I try to retrieve some cities using ***City (wd:Q515)***.

In [30]:
queryString = """
SELECT DISTINCT ?city ?cityName WHERE { 

    # Retrieving cities
    ?city wdt:P31/wdt:P279* wd:Q515.
           
    # This returns the labels
    ?city <http://schema.org/name> ?cityName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q180226'), ('cityName', 'Rancho Palos Verdes')]
[('city', 'http://www.wikidata.org/entity/Q2156427'), ('cityName', 'Destin')]
[('city', 'http://www.wikidata.org/entity/Q712977'), ('cityName', 'Checheng')]
[('city', 'http://www.wikidata.org/entity/Q982554'), ('cityName', 'Coppell')]
[('city', 'http://www.wikidata.org/entity/Q1014001'), ('cityName', 'Montgomery')]
[('city', 'http://www.wikidata.org/entity/Q2076243'), ('cityName', 'Collins')]
[('city', 'http://www.wikidata.org/entity/Q2333965'), ('cityName', 'Chisholm')]
[('city', 'http://www.wikidata.org/entity/Q377736'), ('cityName', 'Bell')]
[('city', 'http://www.wikidata.org/entity/Q576830'), ('cityName', 'Oakbrook Terrace')]
[('city', 'http://www.wikidata.org/entity/Q578778'), ('cityName', 'Northlake')]


10

I want to retrieve only capitals of kingdoms.

In [31]:
queryString = """
SELECT DISTINCT ?kingdomName ?capitalName WHERE { 

    # Retrieve Kingdoms and their capitals
    ?kingdom (wdt:P31|wdt:P122) wd:Q417175 ;
             wdt:P36            ?capital.
    
    # Ensuring that capitals are instances of city
    ?capital wdt:P31/wdt:P279* wd:Q515.
           
    # This returns the labels
    ?kingdom <http://schema.org/name> ?kingdomName .
    ?capital <http://schema.org/name> ?capitalName .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('kingdomName', 'Kingdom of Hungary'), ('capitalName', 'Budapest')]
[('kingdomName', 'Kingdom of Hungary in the Middle Ages'), ('capitalName', 'Esztergom')]
[('kingdomName', 'Eastern Hungarian Kingdom'), ('capitalName', 'Lipova')]
[('kingdomName', 'Kingdom of Wessex'), ('capitalName', 'London')]
[('kingdomName', 'Kingdom of Essex'), ('capitalName', 'London')]
[('kingdomName', 'Luyag na Kaboloan'), ('capitalName', 'San Carlos')]
[('kingdomName', 'Kingdom of the Netherlands'), ('capitalName', 'Amsterdam')]
[('kingdomName', 'Kingdom of Dalmatia'), ('capitalName', 'Split')]
[('kingdomName', 'Kingdom of Saxony'), ('capitalName', 'Dresden')]
[('kingdomName', 'Danish Realm'), ('capitalName', 'Copenhagen')]


10

I can finally retrieve cities that have been capital of multiple kingdoms.

In [32]:
queryString = """
SELECT ?capital ?capitalName COUNT(DISTINCT ?kingdom) AS ?numKingdoms WHERE { 

        # Retrieve kingdoms and their capitals
        ?kingdom (wdt:P31|wdt:P122)  wd:Q417175 ;
                      wdt:P36            ?capital   .

        # Ensuring that capitals are instances of city
        ?capital wdt:P31/wdt:P279* wd:Q515.
            
        # This returns the labels
        ?kingdom <http://schema.org/name> ?kingdomName .
        ?capital <http://schema.org/name> ?capitalName .

}
GROUP BY ?capital ?capitalName
HAVING (COUNT(DISTINCT ?kingdom) > 1)
ORDER BY DESC(?numKingdoms)
 
"""

print("Results")
run_query(queryString)

Results
[('capital', 'http://www.wikidata.org/entity/Q172415'), ('capitalName', 'Kutaisi'), ('numKingdoms', '4')]
[('capital', 'http://www.wikidata.org/entity/Q193478'), ('capitalName', 'Buda'), ('numKingdoms', '3')]
[('capital', 'http://www.wikidata.org/entity/Q994'), ('capitalName', 'Tbilisi'), ('numKingdoms', '2')]
[('capital', 'http://www.wikidata.org/entity/Q1748'), ('capitalName', 'Copenhagen'), ('numKingdoms', '2')]
[('capital', 'http://www.wikidata.org/entity/Q41496'), ('capitalName', 'Thanjavur'), ('numKingdoms', '2')]
[('capital', 'http://www.wikidata.org/entity/Q1530'), ('capitalName', 'Baghdad'), ('numKingdoms', '2')]
[('capital', 'http://www.wikidata.org/entity/Q84'), ('capitalName', 'London'), ('numKingdoms', '2')]
[('capital', 'http://www.wikidata.org/entity/Q1218'), ('capitalName', 'Jerusalem'), ('numKingdoms', '2')]


8

I want also to show the names of the kingdoms using ***GROUP_CONCAT***.

In [33]:
queryString = """
SELECT ?capital ?capitalName COUNT(DISTINCT ?kingdom) AS ?numKingdoms (GROUP_CONCAT(DISTINCT ?kingdomName ; separator = ", ") AS ?kingdoms) WHERE { 

        # Retrieve kingdoms and their capitals
        ?kingdom (wdt:P31|wdt:P122)  wd:Q417175 ;
                      wdt:P36            ?capital   .

        # Ensuring that capitals are instances of city
        ?capital wdt:P31/wdt:P279* wd:Q515.
            
        # This returns the labels
        ?kingdom <http://schema.org/name> ?kingdomName .
        ?capital <http://schema.org/name> ?capitalName .

}
GROUP BY ?capital ?capitalName
HAVING (COUNT(DISTINCT ?kingdom) > 1)
ORDER BY DESC(?numKingdoms)
"""

print("Results")
run_query(queryString)

Results
[('capital', 'http://www.wikidata.org/entity/Q172415'), ('capitalName', 'Kutaisi'), ('numKingdoms', '4'), ('kingdoms', 'Colchis, Kingdom of Abkhazia, Kingdom of Georgia, Kingdom of Imereti')]
[('capital', 'http://www.wikidata.org/entity/Q193478'), ('capitalName', 'Buda'), ('numKingdoms', '3'), ('kingdoms', 'Eastern Hungarian Kingdom, Kingdom of Hungary')]
[('capital', 'http://www.wikidata.org/entity/Q994'), ('capitalName', 'Tbilisi'), ('numKingdoms', '2'), ('kingdoms', 'Kingdom of Georgia, Kingdom of Kartli')]
[('capital', 'http://www.wikidata.org/entity/Q1748'), ('capitalName', 'Copenhagen'), ('numKingdoms', '2'), ('kingdoms', 'Danish Realm, Kalmar Union')]
[('capital', 'http://www.wikidata.org/entity/Q41496'), ('capitalName', 'Thanjavur'), ('numKingdoms', '2'), ('kingdoms', 'Thanjavur Maratha kingdom, Thanjavur Nayak kingdom')]
[('capital', 'http://www.wikidata.org/entity/Q1530'), ('capitalName', 'Baghdad'), ('numKingdoms', '2'), ('kingdoms', 'Buyid dynasty, Kingdom of Iraq')

8

#### END TASK 5.2: 
With this final query I showed cities that have been capital of multiple kingdoms. ***Kutaisi (wd:Q172415)*** is the city that have been capital of the highest number of kingdoms.

### ***Task 5.3 :*** Which are the top-3 monarchies with the earliest inception and still existing?

In a previous query I have already discovered how to retrieve inceptionDate using the property ***inception (wdt:P571)***. 

Now, I need to understand how to ensure that a monarchy is still existing. 

I retrieve another time all the data properties of ***Kingdom of Italy (wd:Q172579)***.

In [34]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Connecting Kingdom of Italy to something
    wd:Q172579  ?p  ?o.

    # This returns the labels
    ?p <http://schema.org/name> ?pName .

    # Only data properties
    FILTER(isLiteral(?o))
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pName', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('pName', 'official name')]
[('p', 'http://www.wikidata.org/prop/direct/P1549'), ('pName', 'demonym')]
[('p', 'http://www.wikidata.org/prop/direct/P1711'), ('pName', 'British Museum person or institution ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pName', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P2046'), ('pName', 'area')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4839'), ('pName', 'Wolfram Language entity code')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'in

20

Probably, I can use the property ***dissolved, abolished or demolished date (wdt:P576)*** to actually check if a monarchy is still existing. I try to use this property.

In [35]:
queryString = """
SELECT ?dissolutionDate WHERE { 

    # Retrieve the dissolutionDate of Kingdom of Italy
    wd:Q172579  wdt:P576  ?dissolutionDate.

}
"""

print("Results")
run_query(queryString)

Results
[('dissolutionDate', '1946-06-18T00:00:00Z')]


1

I retrieve the inception date of the monarchies (using ***inception (wdt:P571)***) that are still existing, so the ones that do not have the property ***dissolved, abolished or demolished date (wdt:P576)***.

In [36]:
queryString = """
SELECT DISTINCT ?monarchy ?monarchyName ?inceptionDate WHERE { 

    # Retrieve monarchies and their inceptionDate
    ?monarchy   (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269       ; 
                wdt:P571                      ?inceptionDate .

    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
    
    # Ensuring that monarchies are still existing
    FILTER NOT EXISTS { ?monarchy wdt:P576 ?dissolutionDate }
}
ORDER BY ASC(?inceptionDate)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q2349630'), ('monarchyName', 'Galactic Empire'), ('inceptionDate', '11584-01-01T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q787336'), ('monarchyName', 'Roman Catholic Diocese of Metz'), ('inceptionDate', '0201-01-01T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q42834'), ('monarchyName', 'Western Roman Empire (395-476 AD)'), ('inceptionDate', '0395-01-18T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q384237'), ('monarchyName', 'duchy of Asti'), ('inceptionDate', '0569-01-01T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q1249360'), ('monarchyName', 'Roman Catholic Diocese of Bolzano-Brixen'), ('inceptionDate', '0600-01-01T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q34'), ('monarchyName', 'Sweden'), ('inceptionDate', '0700-01-01T00:00:00Z')]
[('monarchy', 'http://www.wikidata.org/entity/Q756617'), ('monarchyName', 'Danish Realm'), ('inceptionDate', '0800-01-01T00:0

10

There are some strange reesult so I start to inspect them using ***instanceOf (wdt:P31)***, one by one starting from the oldest one.

First, I have ***Galactic Empire (wd:Q2349630)***.

In [37]:
queryString = """
SELECT ?class ?className WHERE { 

    # Connecting Galactic Empire using instanceOf
    wd:Q2349630  wdt:P31 ?class .

    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q653007'), ('className', 'galactic empire')]
[('class', 'http://www.wikidata.org/entity/Q19595246'), ('className', 'fictional empire')]


2

The ***Galactic Empire (wd:Q2349630)*** is a fictional empire, so I have to exclude it.

It's time for ***Roman Catholic Diocese of Metz (wd:Q787336)***.

In [38]:
queryString = """
SELECT ?class ?className WHERE { 

    # Connecting Roman Catholic Diocese of Metz using instanceOf
    wd:Q787336  wdt:P31 ?class .

    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q2665272'), ('className', 'Ecclesiastical circumscription immediately subject to the Holy See')]
[('class', 'http://www.wikidata.org/entity/Q3403564'), ('className', 'ecclesiastical principality')]
[('class', 'http://www.wikidata.org/entity/Q3146899'), ('className', 'diocese of the Catholic Church')]


3

The ***Roman Catholic Diocese of Metz (wd:Q787336)*** (and probably also Roman Catholic Diocese of Bolzano-Brixen ) is an ***ecclesiastical principality (wd:Q3403564)***. 

Since principality is a specific sub-type of monarchy (as I discovered in a previous query) it is also considered a monarchy.

In my opinion, I should not consider it since nowadays dioceses can not be considered monarchies. Indeed, they can be considered more like territorial jurisdictions rather than "real countries".

Now I try to discover more about ***Western Roman Empire (395-476 AD) (wd:Q42834)*** .

In [39]:
queryString = """
SELECT ?class ?className WHERE { 

    # Connecting Western Roman Empire  using instanceOf
    wd:Q42834  wdt:P31 ?class .

    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3024240'), ('className', 'historical country')]
[('class', 'http://www.wikidata.org/entity/Q48349'), ('className', 'empire')]


2

The Western Roman Empire (395-476 AD)  is of course a real empire, but probably ***dissolved, abolished or demolished date (wdt:P576)*** is missing. I show all the properties.

In [40]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve all the properties of Western Roman Empire 
    wd:Q42834  ?p ?o .

    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pName', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pName', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('pName', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1667'), ('pName', 'Getty Thesaurus of Geographic Names ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P194'), ('pName', 'legislative body')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName

36

The property ***dissolved, abolished or demolished date (wdt:P576)*** is missing. 

I can notice that Western Roman Empire (395-476 AD) was actually replaced by another state (due to the presence of the property ***replacedBy (wdt:P1366)***, so I should not consider it.

I inspect the ***duchy of Asti (wd:Q384237)***.

In [41]:
queryString = """
SELECT ?class ?className WHERE { 

    # Connecting duchy of Asti sing instanceOf
    wd:Q384237 wdt:P31 ?class .

    # This returns the labels
    ?class <http://schema.org/name> ?className .
}
"""

print("Results")
run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q154547'), ('className', 'duchy')]


1

The ***duchy of Asti (wd:Q384237)*** seems to be a real monarchy but maybe ***dissolved, abolished or demolished date (wdt:P576)*** is missing. I show all the properties.

In [42]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve all the properties of duchy of Asti
    wd:Q384237  ?p ?o .

    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('pName', 'capital')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception')]


4

In this case the information about dissolutionDate is missing, so I should exclude it.

I excluded all the strange cases. The answer to the initial question should focus on these: Sweden, Danish Realm, Denmark, Norway.

I want to check if there is any property between ***Danish Realm (wd:Q756617)*** and ***Denmark (wd:Q35)***.

In [43]:
queryString = """
SELECT DISTINCT ?p ?pName WHERE { 

    # Retrieve all the properties between Danish Realm and Denmark
    wd:Q756617 ?p wd:Q35 .
    
    # This returns the labels
    ?p <http://schema.org/name> ?pName .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]


3

Due to the property ***hasPart (wdt:P527)***  I can consider ***Denmark (wd:Q35)*** as a part of the ***Danish Realm (wd:Q756617)***.

#### END TASK 5.2: 

I can finally answer to the initial question. The top-3 monarchies with the earliest inception and still existing are:
* ***Sweden*** with inceptionDate = ***'0700-01-01T00:00:00Z'***
* ***Danish Realm*** with inceptionDate = ***'0800-01-01T00:00:00Z'***
* ***Norway*** with inceptionDate = ***'0900-01-01T00:00:00Z'***

In my opinion, it is difficult to write a single query to answer this question, without inspecting one by one all the strange cases. Indeed, even if it is actually possibile to remove "fictional empire", the other cases were due to some missing information or due to the fact that ecclesiastic dioceses nowadays should be not considered monarchies.

### Extra analytics query

#### Show the average population of countries among the different continents.

In [44]:
queryString = """
SELECT ?continent ?continentName AVG(xsd:integer(?population)) AS ?avgPopulationByContinent WHERE {
        
    # Retrieve countries and their population and also their continent
    ?country   wdt:P31     wd:Q6256    ;
               wdt:P1082   ?population ;
               wdt:P30     ?continent  .

    # I want only "traditional" continent
    ?continent wdt:P31 wd:Q5107 .
            
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName .
}
GROUP BY ?continent ?continentName
ORDER BY DESC(?avgPopulationByContinent)
"""

print("Results")
run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('avgPopulationByContinent', '100504008.58695652173913')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('avgPopulationByContinent', '35175756.5')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('avgPopulationByContinent', '33846238.894736842105263')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('avgPopulationByContinent', '24971270.833333333333333')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('avgPopulationByContinent', '24154589.769230769230769')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('avgPopulationByContinent', '14890442.8')]


6

#### Show the three most populated monarchies with dissolutionDate before 1700.

In [45]:
queryString = """
SELECT DISTINCT ?monarchyName ?population ?dissolutionDate WHERE {
        
            # Retrieve monarchies, their population and also their dissolutionDate
            ?monarchy (wdt:P31|wdt:P122)/wdt:P279*   wd:Q7269         ;
                       wdt:P1082                     ?population      ;
                       wdt:P576                      ?dissolutionDate .
                    

            # This returns the labels
            ?monarchy <http://schema.org/name> ?monarchyName .
            
            # Only monarchies with dissolution date before 1700
            FILTER (?dissolutionDate < "1700-01-01T00:00:00Z"^^xsd:dateTime)
}
ORDER BY DESC(?population)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('monarchyName', 'Mongol Empire'), ('population', '160000000'), ('dissolutionDate', '1368-01-01T00:00:00Z')]
[('monarchyName', 'Umayyad Caliphate'), ('population', '70000000'), ('dissolutionDate', '0750-01-01T00:00:00Z')]
[('monarchyName', 'Han dynasty'), ('population', '59594978'), ('dissolutionDate', '0220-01-01T00:00:00Z')]


3

#### Show average population of monarchies with dissolutionDate before 1700 among different continents.

In [46]:
queryString = """
SELECT ?continent ?continentName AVG(xsd:integer(?population)) AS ?avgPopulationMonarchies WHERE {

    # Retrieve monarchies, their population and also their dissolutionDate
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279*  wd:Q7269         ;
              wdt:P1082                     ?population      ;
              wdt:P576                      ?dissolutionDate ;
              wdt:P30                       ?continent       .

    # I want only "traditional" continent
    ?continent wdt:P31    wd:Q5107    .

    # Only monarchies with dissolution date before 1700
    FILTER (?dissolutionDate < "1700-01-01T00:00:00Z"^^xsd:dateTime)
                
    # This returns the labels
    ?continent <http://schema.org/name> ?continentName .
}
GROUP BY ?continent ?continentName
ORDER BY DESC(?avgPopulationMonarchies)

"""

print("Results")
run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('avgPopulationMonarchies', '42333333.333333333333333')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('avgPopulationMonarchies', '30186870.444444444444444')]
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('avgPopulationMonarchies', '16900000')]
[('continent', 'http://www.wikidata.org/entity/Q828'), ('continentName', 'Americas'), ('avgPopulationMonarchies', '1500000')]


4

#### Show monarchies with the highest number of capitals.

In [47]:
queryString = """
SELECT ?monarchy ?monarchyName (COUNT(DISTINCT ?capital) AS ?numCapitals) WHERE { 

    # Retrieve monarchies and their capitals
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279*   wd:Q7269 ;
               wdt:P36                       ?capital .

    # Ensuring that capitals are instances of city
    ?capital   wdt:P31/wdt:P279*             wd:Q515  .   
            
    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
}
GROUP BY ?monarchy ?monarchyName
HAVING (COUNT(DISTINCT ?capital) > 1)
ORDER BY DESC(?numCapitals)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q12548'), ('monarchyName', 'Holy Roman Empire'), ('numCapitals', '5')]
[('monarchy', 'http://www.wikidata.org/entity/Q3708094'), ('monarchyName', 'Seljuk Empire'), ('numCapitals', '4')]
[('monarchy', 'http://www.wikidata.org/entity/Q33296'), ('monarchyName', 'Mughal Empire'), ('numCapitals', '4')]
[('monarchy', 'http://www.wikidata.org/entity/Q45670'), ('monarchyName', 'Kingdom of Portugal'), ('numCapitals', '4')]
[('monarchy', 'http://www.wikidata.org/entity/Q22880'), ('monarchyName', 'Electoral Palatinate'), ('numCapitals', '3')]
[('monarchy', 'http://www.wikidata.org/entity/Q193619'), ('monarchyName', 'Union of South Africa'), ('numCapitals', '3')]
[('monarchy', 'http://www.wikidata.org/entity/Q188586'), ('monarchyName', 'Kingdom of Sicily'), ('numCapitals', '3')]
[('monarchy', 'http://www.wikidata.org/entity/Q200464'), ('monarchyName', 'Portuguese Empire'), ('numCapitals', '3')]
[('monarchy', 'http://www.wikidata.org/entity/Q272

10

#### Show also the names of these capitals.

In [48]:
queryString = """
SELECT ?monarchy ?monarchyName (COUNT(DISTINCT ?capital) AS ?numCapitals) (GROUP_CONCAT(DISTINCT ?capitalName ; separator = ", ") AS ?capitals) WHERE { 

    # Retrieve monarchies and their capitals
    ?monarchy (wdt:P31|wdt:P122)/wdt:P279*   wd:Q7269 ;
               wdt:P36                       ?capital .

    # Ensuring that capitals are instances of city
    ?capital   wdt:P31/wdt:P279*             wd:Q515  .   
            
    # This returns the labels
    ?monarchy <http://schema.org/name> ?monarchyName .
    ?capital  <http://schema.org/name> ?capitalName .
}
GROUP BY ?monarchy ?monarchyName
HAVING (COUNT(DISTINCT ?capital) > 1)
ORDER BY DESC(?numCapitals)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('monarchy', 'http://www.wikidata.org/entity/Q12548'), ('monarchyName', 'Holy Roman Empire'), ('numCapitals', '5'), ('capitals', 'Aachen, Prague, Regensburg, Vienna, Wetzlar')]
[('monarchy', 'http://www.wikidata.org/entity/Q3708094'), ('monarchyName', 'Seljuk Empire'), ('numCapitals', '4'), ('capitals', 'Hamadan, Isfahan, Nishapur, Ray')]
[('monarchy', 'http://www.wikidata.org/entity/Q33296'), ('monarchyName', 'Mughal Empire'), ('numCapitals', '4'), ('capitals', 'Agra, Delhi, Lahore, Old Delhi')]
[('monarchy', 'http://www.wikidata.org/entity/Q45670'), ('monarchyName', 'Kingdom of Portugal'), ('numCapitals', '4'), ('capitals', 'Coimbra, Guimarães, Lisbon, Rio de Janeiro')]
[('monarchy', 'http://www.wikidata.org/entity/Q22880'), ('monarchyName', 'Electoral Palatinate'), ('numCapitals', '3'), ('capitals', 'Düsseldorf, Heidelberg, Mannheim')]
[('monarchy', 'http://www.wikidata.org/entity/Q193619'), ('monarchyName', 'Union of South Africa'), ('numCapitals', '3'), ('capitals', 'Bloe

10

#### Show the average number of capitals for monarchies with at least 2 capitals

In [49]:
queryString = """
SELECT AVG(?numCapitals) AS ?avgNumCapitals WHERE{
 
    {   SELECT ?monarchy (COUNT(DISTINCT ?capital) AS ?numCapitals) WHERE { 

            # Retrieve monarchies and their capitals
            ?monarchy (wdt:P31|wdt:P122)/wdt:P279*   wd:Q7269 ;
                       wdt:P36                       ?capital .

            # Ensuring that capitals are instances of city
            ?capital   wdt:P31/wdt:P279*             wd:Q515  .     
            
            
            # This returns the labels
            ?monarchy <http://schema.org/name> ?monarchyName .
        }
        GROUP BY ?monarchy
        HAVING (COUNT(DISTINCT ?capital) > 1)
        }    
}
"""

print("Results")
run_query(queryString)

Results
[('avgNumCapitals', '2.301075268817204')]


1

##  END